# Chiffrement de vigenère

## Chiffrement

In [1]:
def encrypt(plain, key,alphabet=26):
    output = ""
    key= key.upper()
    plain= plain.upper()
    
    for i, s in enumerate(plain):
        #print(i,"   ",s, "   ", key[i%len(key)])
        output += chr((ord(s) + ord(key[i%len(key)]) - 2*ord("A")) % alphabet + ord("A"))
    return output

In [2]:
encrypt("IKHLEFALI","HELLO")

'POSWSMEWT'

## Déchiffrement

In [3]:
def decrypt(cipher, key,alphabet=26):
    output = ""
    key = key.upper()
    cipher = cipher.upper()
    
    for i, s in enumerate(cipher):
        #print(i,"   ",s, "   ", key[i%len(key)])
        output += chr((ord(s) - ord(key[i%len(key)]) - 2*ord("A")) % alphabet + ord("A"))
    return output

In [4]:
decrypt("POSWSMEWT","HELLO")

'IKHLEFALI'

# Cryptanalyse de cet algorithme

## Attaque par force brute

In [5]:
from itertools import product

cipher = encrypt(
    "BEFOREEVERYFEELINGOFTHESUBJECTITMUSTTHATTHESELFEXIST",
    "LINA"
)

word   = "THESELF"
keyOf  = 4
alphabet = "AZERTYUIOPQSDFGHJKLMWXCVBN"


def generate_possible_key(alphabet, lenOfKey):
    keys = []
    for repeat in range(1,lenOfKey + 1 ):
        for k in product(alphabet, repeat=repeat):
            key = ""
            key = key.join(list(k))
            keys.append(key)

    #print(keys)
    return keys

In [6]:
import time

keys = generate_possible_key(alphabet=alphabet,lenOfKey=4)

t0= time.perf_counter()
for i, key in enumerate(keys):
    possible_plain = decrypt(
        cipher,
        key
    )
    if word in possible_plain :
        print(f"Time elapsed:{time.perf_counter() - t0} s")
        print(f"key is : >> {key} <<, after testing {i} keys from {len(keys)} keys ")
        print(f"plain txt is : {possible_plain}")
        break

Time elapsed:9.781473335000555 s
key is : >> LINA <<, after testing 340028 keys from 475254 keys 
plain txt is : BEFOREEVERYFEELINGOFTHESUBJECTITMUSTTHATTHESELFEXIST


## Crypte Analyse

In [7]:
def coincidence(cipher,alphabet=26):
    cipher = cipher.upper()
    freq = alphabet * [0]
    ic = 0
    l = len(cipher)
    for c in cipher:
        freq[ord(c) - 65]+=1
        
    for item in freq:
        ic += item * (item - 1)
    ic /= ( l * (l - 1))
    return ic

In [8]:
cipher = encrypt(
    "BEFOREEVERYFEELINGOFTHESUBJECTITMUSTTHATTHESELFEXIST",
    "LINA"
)
print(cipher)
coincidence(cipher)

MMSOCMRVPZLFPMYIYOBFEPRSFJWENBVTXCFTEPNTEPRSPTSEIQFT


0.05052790346907994

### Determination de la taille de la clé

In [9]:
MAX_LENGHT = 25

In [10]:
def get_key_len(cipher):
    cipher = cipher.upper()
    ics = []
    for probable_len in range(1,MAX_LENGHT):
        ic     = 0.0
        avg_ic = 0.0
        for i in range(0,probable_len):
            subcipher = ""
            for j in range(0, len(cipher[i:]),probable_len):
                subcipher += cipher[i+j]
            #print(subcipher)
            ic += coincidence(subcipher)
        
        avg_ic=ic/probable_len
        ics.append((avg_ic,probable_len))
    
    # Za3ma
    for idx, v in enumerate(sorted(ics, reverse = True, key= lambda item: item[0])):
        if v[0] < 0.1:
            p1 = sorted(ics, reverse = True, key= lambda item: item[0])[idx]
            p2 = sorted(ics, reverse = True, key= lambda item: item[0])[idx+1]
            break
    
    if p1[1] % p2[1] == 0:
        return p2
    else :
        return p1

In [11]:
get_key_len("MMSOCMRVPZLFPMYIYOBFEPRSFJWENBVTXCFTEPNTEPRSPTSEIQFT")

(0.08333333333333334, 4)

### Determination de la clé

http://pi.math.cornell.edu/~mec/2003-2004/cryptography/subs/frequencies.html

In [12]:
# Za3ma :/
english_freqs = [8.12,1.49,2.71,4.32,12.02,
 2.30,0.10,5.92,7.31,2.03,
 0.69,3.98,2.61,6.95,7.68,
 1.82,0.11,6.02,6.28,9.10,
 2.88,1.11,0.17,2.09,2.11,0.07,
]

In [13]:
def freq_analysis(cipher,reference,alphabet=26):
    cipher = cipher.upper()
    l =  len(cipher)
    freqs = alphabet * [0]
    
    for c in cipher:
        freqs[ord(c) - ord("A")] += (1/l)*100
        
    # calculate given func
    d = 0.0
    for i in range(alphabet):
        d+= abs(freqs[i] - reference[i])
        
    ref_max = reference.index(max(reference))
    freq_max = freqs.index(max(freqs))
    
    return (chr((freq_max - ref_max)%alphabet + ord("A")),d/100)

In [14]:
def get_key(cipher,lenOfkey):
    key = ""
    
    for i in range(lenOfkey):
        seq = ""
        for j in range(0,len(cipher[i:]),lenOfkey):
            seq+=cipher[i+j]
        key+=freq_analysis(seq,english_freqs)[0]
    print(key)

In [15]:
get_key(
    cipher="MMSOCMRVPZLFPMYIYOBFEPRSFJWENBVTXCFTEPNTEPRSPTSEIQFT",
    lenOfkey=4
)

AINP
